In [3]:
import numpy as np
import os
import torch
from torch.utils.data import DataLoader, Dataset
FLUX_DATA_DIR = 'C:\\Users\\Alberto\\Desktop\\uni_tmp\\ADL\\STACIE\\flux_prediction\\data\\magnetic_flux_area_data\\'
LSTM_DATA_DIR = 'C:\\Users\\Alberto\\Desktop\\uni_tmp\\ADL\\STACIE\\flux_prediction\\data\\lstm_data\\multi_label_classification\\'

In [2]:
file_inp = LSTM_DATA_DIR + 'train_input.npy'
file_tar = LSTM_DATA_DIR + 'train_target.npy'

train_input = np.load(file_inp)
train_target = np.load(file_tar)

file2_inp = LSTM_DATA_DIR + 'val_input.npy'
file2_tar = LSTM_DATA_DIR + 'val_target.npy'

val_input = np.load(file2_inp)
val_target = np.load(file2_tar)

In [8]:
ds = DataLoader(train_input, batch_size=256)
len(ds)

304

In [16]:
 def split_classes(targets, n_classes=5):
    index_list = [[] for _ in range(n_classes)]
    for i, tgt in enumerate(targets):
        class_idx = np.argmax(tgt[0])
        index_list[class_idx].append(i)
    for class_num in index_list:
        print(len(class_num))
    return index_list

def downsample_data(sources, targets):
    class_idx = split_classes(targets)
    min_class = min([len(label) for label in class_idx])
    out_idx = []
    for class_list in class_idx:
        if len(class_list) > min_class:
            idx_list = np.random.choice(class_list, size=min_class, replace=False)
            out_idx.extend(idx_list)
        else:
            out_idx.extend(class_list)
    return sources[out_idx], targets[out_idx]

def split_flare(targets):
    noflare_idx = []
    flare_idx = []
    for i, tgt in enumerate(targets):
        class_idx = np.argmax(tgt[0])
        if class_idx == 0:
            noflare_idx.append(i)
        else:
            flare_idx.append(i)
    print(len(noflare_idx), len(flare_idx))
    return noflare_idx, flare_idx

In [17]:
#idx = split_classes(train_target)
#print(idx[-1])
#class_5_tgt = train_target[idx[-1]]
#print(class_5_tgt)
new_sources, new_targets = downsample_data(train_input, train_target)
print(len(new_sources), len(new_targets), 226*5)

51614
14614
8963
2321
226
1130 1130 1130


In [27]:
def label_weights(targets):
    class_num = np.zeros(5)
    print(len(targets))
    for tgt in targets:
        label_idx = np.argmax(tgt[0])
        class_num[label_idx] += 1
    class_num = class_num/len(targets)
    weights = np.ones(5) - class_num
    return class_num, weights

In [29]:
c, w = label_weights(train_target)
print(c)
print(w)

77738
[0.66394813 0.18799043 0.11529754 0.0298567  0.0029072 ]
[0.33605187 0.81200957 0.88470246 0.9701433  0.9970928 ]


In [5]:
train_input.shape

(77738, 10, 40)

In [6]:
val_input.shape

(24683, 10, 40)

In [13]:
new = np.concatenate((train_input, val_input))

In [14]:
new.shape

(102421, 10, 40)

In [12]:
#for n in np.argmax(train_target, axis=2):
#    print(n, ' ', end='')

a = np.argmax(train_target, axis=2).flatten()
print(a)

[0 0 0 ... 1 1 1]


In [28]:
t2_input = train_input
t2_output = train_target

In [29]:
my_shape = t2_input.shape
#my_shape = (my_shape[0], my_shape[1], my_shape[2] + 5)
my_shape2 = t2_output.shape

print(my_shape, my_shape2)

(77738, 10, 40) (77738, 1, 5)


In [30]:
array1 = np.pad(t2_input, ((0,0), (0,0), (0,5)), mode='constant')
array2 = np.pad(t2_output, ((0,0), (0,0), (40,0)), mode='constant')

learning rate tutorial: https://pytorch-forecasting.readthedocs.io/en/stable/tutorials/stallion.html

In [34]:
array2[500][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [13]:
train_target[100,0,0]

1.0

In [15]:
import torch
from sklearn.metrics import f1_score

In [3]:
t = torch.Tensor([[ 0.0825,  0.1514],
        [ 1.0017, -0.9427],
        [-0.2009,  0.5858],
        [ 0.5714, -0.4565],
        [ 1.1508, -1.1926],
        [ 1.2249, -1.2643],
        [ 1.0297, -1.0032],
        [ 0.1368,  0.1709],
        [ 0.8833, -0.8309],
        [-0.0853,  0.2499],
        [ 1.2265, -1.2521],
        [ 1.1567, -1.2178],
        [ 0.4999, -0.3505],
        [ 1.2842, -1.3214],
        [ 1.0780, -1.1057],
        [ 0.4917, -0.2947],
        [ 0.3241, -0.0980],
        [ 0.8195, -0.7421],
        [ 0.9674, -1.0017],
        [ 0.3735, -0.2303],
        [ 0.4533, -0.2839],
        [ 0.2105,  0.0082],
        [ 0.6989, -0.7105],
        [-0.7834,  0.8882],
        [ 0.0805,  0.1199],
        [ 0.2665,  0.0696],
        [ 0.8326, -0.7673],
        [ 0.3153, -0.0355],
        [ 1.0380, -1.1166],
        [ 0.8410, -0.7189],
        [ 0.2861, -0.2109],
        [ 0.9142, -0.8246],
        [ 0.1956,  0.1142],
        [ 0.2774,  0.0247],
        [ 0.5453, -0.3120],
        [ 1.1921, -1.3566],
        [ 0.1720, -0.0827],
        [ 1.0999, -1.1057],
        [ 1.0848, -1.1121],
        [ 0.3071, -0.1220],
        [ 0.4445, -0.3426],
        [ 1.0167, -0.9883],
        [ 0.8781, -0.6220]])

In [9]:
dim=2
            
label_idx = torch.argmax(t, dim=1)
print(label_idx)

#label_idx = label_idx[:, 0]
#print(label_idx)

tensor([1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


IndexError: too many indices for tensor of dimension 1

In [16]:
from typing import Tuple

import torch


class F1Score:
    """
    Class for f1 calculation in Pytorch.
    """

    def __init__(self, average: str = 'weighted'):
        """
        Init.

        Args:
            average: averaging method
        """
        self.average = average
        if average not in [None, 'micro', 'macro', 'weighted']:
            raise ValueError('Wrong value of average parameter')

    @staticmethod
    def calc_f1_micro(predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 micro.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """
        true_positive = torch.eq(labels, predictions).sum().float()
        f1_score = torch.div(true_positive, len(labels))
        return f1_score

    @staticmethod
    def calc_f1_count_for_label(predictions: torch.Tensor,
                                labels: torch.Tensor, label_id: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Calculate f1 and true count for the label

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels
            label_id: id of current label

        Returns:
            f1 score and true count for label
        """
        # label count
        true_count = torch.eq(labels, label_id).sum()

        # true positives: labels equal to prediction and to label_id
        true_positive = torch.logical_and(torch.eq(labels, predictions),
                                          torch.eq(labels, label_id)).sum().float()
        # precision for label
        precision = torch.div(true_positive, torch.eq(predictions, label_id).sum().float())
        # replace nan values with 0
        precision = torch.where(torch.isnan(precision),
                                torch.zeros_like(precision).type_as(true_positive),
                                precision)

        # recall for label
        recall = torch.div(true_positive, true_count)
        # f1
        f1 = 2 * precision * recall / (precision + recall)
        # replace nan values with 0
        f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1).type_as(true_positive), f1)
        return f1, true_count

    def __call__(self, predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 score based on averaging method defined in init.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """

        # simpler calculation for micro
        if self.average == 'micro':
            return self.calc_f1_micro(predictions, labels)

        f1_score = 0
        for label_id in range(len(labels.unique())):
            f1, true_count = self.calc_f1_count_for_label(predictions, labels, label_id)

            if self.average == 'weighted':
                f1_score += f1 * true_count
            elif self.average == 'macro':
                f1_score += f1

        if self.average == 'weighted':
            f1_score = torch.div(f1_score, len(labels))
        elif self.average == 'macro':
            f1_score = torch.div(f1_score, len(labels.unique()))

        return f1_score

In [17]:
labels = torch.randint(0, 10, (4096, 100)).flatten()
predictions = torch.randint(0, 10, (4096, 100)).flatten()

In [19]:
predictions

tensor([7, 3, 1,  ..., 5, 4, 7])

In [20]:

labels1 = labels.numpy()
predictions1 = predictions.numpy()

f1_metric = F1Score('macro')
my_pred = f1_metric(predictions, labels)

f1_pred = f1_score(labels1, predictions1, average='macro')

print(my_pred, f1_pred)

tensor(0.1007) 0.10068788783404362


In [4]:
def f1(y_true, y_pred):
    EPSILON = 1e-7
    y_pred = torch.round(y_pred)
    a = y_true*y_pred
    a.type(torch.FloatTensor)
    tp = torch.sum(a, axis=0)
    
    b = (1-y_true)*(1-y_pred)
    b.type(torch.FloatTensor)
    tn = torch.sum(b, axis=0)
    
    c = (1-y_true)*y_pred
    c.type(torch.FloatTensor)
    fp = torch.sum(c, axis=0)
    
    d = y_true*(1-y_pred)
    d.type(torch.FloatTensor)
    fn = torch.sum(d, axis=0)

    p = tp / (tp + fp + EPSILON)
    r = tp / (tp + fn + EPSILON)

    f1 = 2*p*r / (p+r+EPSILON)
    f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)
    return torch.mean(f1)

"""
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
"""

"\ndef f1_loss(y_true, y_pred):\n    \n    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)\n    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)\n    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)\n    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)\n\n    p = tp / (tp + fp + K.epsilon())\n    r = tp / (tp + fn + K.epsilon())\n\n    f1 = 2*p*r / (p+r+K.epsilon())\n    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)\n    return 1 - K.mean(f1)\n"

In [31]:
NHEADS=4
MODEL_DIM=64
ENC_LAYERS=4
ENCODER='pos'
loss_str='cross entropy'
OPTIM='adam'
lr=2e-5
BATCH_SIZE=16
EPOCHS=20
DEV='cuda:0'
OUT_NAME='cagasbura'
print(f'Heads : {NHEADS}\nDimension : {MODEL_DIM}\nEncoding layers : {ENC_LAYERS}')

print(f'Encoder : {ENCODER}\nLoss function : {loss_str}\nOptimizer : {OPTIM}\nLearning rate : {lr}')
print(f'Batch size : {BATCH_SIZE}\nEpochs : {EPOCHS}\nDevice : {DEV}\nOutput name : {OUT_NAME}')

Heads : 4
Dimension : 64
Encoding layers : 4
Encoder : pos
Loss function : cross entropy
Optimizer : adam
Learning rate : 2e-05
Batch size : 16
Epochs : 20
Device : cuda:0
Output name : cagasbura
